In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "DT_node_data" AS 
(SELECT "Values".node_id AS node_id, "Values".feature AS feature, "Values".threshold AS threshold, "Values".count AS count, "Values".depth AS depth, "Values"."Proba_0.0" AS "Proba_0.0", "Values"."LogProba_0.0" AS "LogProba_0.0", "Values"."Proba_1.0" AS "Proba_1.0", "Values"."LogProba_1.0" AS "LogProba_1.0", "Values"."Proba_2.0" AS "Proba_2.0", "Values"."LogProba_2.0" AS "LogProba_2.0", "Values"."Decision" AS "Decision" 
FROM (SELECT 0 AS node_id, 'Feature_0' AS feature, 5.449999809265137 AS threshold, 93 AS count, 0 AS depth, 0.28 AS "Proba_0.0", -1.27296567581 AS "LogProba_0.0", 0.35333333333333333 AS "Proba_1.0", -1.04034338054 AS "LogProba_1.0", 0.36666666666666664 AS "Proba_2.0", -1.00330210886 AS "LogProba_2.0", 2 AS "Decision" UNION ALL SELECT 1 AS node_id, 'Feature_2' AS feature, 2.5999999046325684 AS threshold, 30 AS count, 1 AS depth, 0.8695652173913043 AS "Proba_0.0", -0.139761942375 AS "LogProba_0.0", 0.10869565217391304 AS "Proba_1.0", -2.21920348405

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
6,6,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
121,121,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
58,58,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
55,55,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
98,98,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
16,16,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
30,30,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
57,57,None,None,None,0.0,0.8,0.2,-1.797693e+308,-2.231436e-01,-1.609438e+00,1
63,63,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
14,14,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0


In [10]:
sql_output.Decision.value_counts()

1    51
0    50
2    49
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12)


/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:628: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
102,102,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2
133,133,NaN,NaN,NaN,0.0,0.3,0.7,-inf,-1.203973,-0.356675,2
98,98,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,1
36,36,NaN,NaN,NaN,0.9,0.1,0.0,-0.105361,-2.302585,-inf,0
77,77,NaN,NaN,NaN,0.0,0.8,0.2,-inf,-0.223144,-1.609438,1
52,52,NaN,NaN,NaN,0.0,0.9,0.1,-inf,-0.105361,-2.302585,1
142,142,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2
121,121,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,2
15,15,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,0
1,1,NaN,NaN,NaN,0.9,0.0,0.1,-0.105361,-inf,-2.302585,0


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
4,4,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,...,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
109,109,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
82,82,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,...,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
134,134,NaN,NaN,NaN,0.0,0.3,0.7,-inf,-1.203973,-0.356675,...,None,None,None,0.0,0.3,0.7,-1.797693e+308,-1.203973e+00,-3.566749e-01,2
105,105,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
71,71,NaN,NaN,NaN,0.0,1.0,0.0,-inf,0.000000,-inf,...,None,None,None,0.0,1.0,0.0,-1.797693e+308,0.000000e+00,-1.797693e+308,1
110,110,NaN,NaN,NaN,0.0,0.1,0.9,-inf,-2.302585,-0.105361,...,None,None,None,0.0,0.1,0.9,-1.797693e+308,-2.302585e+00,-1.053605e-01,2
102,102,NaN,NaN,NaN,0.0,0.0,1.0,-inf,-inf,0.000000,...,None,None,None,0.0,0.0,1.0,-1.797693e+308,-1.797693e+308,0.000000e+00,2
35,35,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,-inf,-inf,...,None,None,None,1.0,0.0,0.0,0.000000e+00,-1.797693e+308,-1.797693e+308,0
138,138,NaN,NaN,NaN,0.0,0.1,0.9,-inf,-2.302585,-0.105361,...,None,None,None,0.0,0.1,0.9,-1.797693e+308,-2.302585e+00,-1.053605e-01,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
